$$
\def\I{{\rm i}}
\def\qqqquad{\qquad\qquad}
\def\qfor{\qquad\hbox{for}\qquad}
\def\qwhere{\qquad\hbox{where}\qquad}
\def\ale#1{\begin{align}#1\end{align}}
\def\em{\epsilon_m}
\def\unit#1{{\rm #1}}
\def\com#1{\color{blue}{\textrm{#1}}} 
$$
# Lab4: Coupled Oscillators (1 week)

- Submit your lab by dropping this notebook with your answers at the online `Dropbox` link provide on slack. 

- __Rename your notebook to__ `Lab4_365yourLoginName.ipynb` before submitting it. 

- If you want to resubmit, past the notebook into a direct message to me on slack.

- __Due:__ 3pm Wednesday 22nd April. Note this lab is not due until after the mid-semester break.

$\com{Please leave this command for marker's comments}$

---

In [1]:
# Imports
import Pkg;
Pkg.add("Plots");
Pkg.add("OrdinaryDiffEq");
Pkg.add("LinearAlgebra");
Pkg.add("Interact");
Pkg.add("LaTeXStrings");

using Plots;
using OrdinaryDiffEq;
using LinearAlgebra;
using Interact;
using SparseArrays;
using LaTeXStrings;

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
 Resolving package versions...
  Updating `~/.julia/environments/v1.3/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.3/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.3/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.3/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.3/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.3/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.3/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.3/Manifest.toml`
 [no changes]
 Resolving package versions...


HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-18264256763912020599\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-9913694610278335165\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-18264256763912020599\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

  Updating `~/.julia/environments/v1.3/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.3/Manifest.toml`
 [no changes]


# Preliminaries
The interactive plots of the displacement for the different scenarios (with and without a driving force) can be generated using the same code.
```julia
function burj_khalifa(parameters, initialConditions, timespan, boundaryCondition, forcing)
    ω, γ, N, B, Ω = parameters; # Define the parameters
    N = convert(Integer, N); # Convert to an Integer
    floors = collect(1:1:N);
        
    A = diagm(0=>2*ones(N), 1=>ones(N-1), -1=>ones(N-1)); # Generate a tri-diagonal matrix
    # Set the boundary conditions
    A[1, 1] = boundaryCondition[1] == true ? 1 : A[1, 1]; # For the first mass
    A[end, end] = boundaryCondition[2] == true ? 1 : A[end, end]; # For the last mass
    
    global L = [zero(A) one(A); -ω^2*A -γ*A] |> sparse; # Generate L and make it sparse to increase speed
    forcing_function(t) =  ω^2*B*sin(Ω*t)+γ*Ω*B*cos(Ω*t); # Forcing function
    F(t) = [zeros(N); forcing_function(t); zeros(N-1)]; # Forcing function vector
    # Model our system and solve it, if there is a driving force, then add it
    function f(du, u, p, t) 
        mul!(du, L, u);
        du .+= forcing == true ? F(t) : 0;
    end
    prob = ODEProblem(f, initialConditions, timespan);
    sol = solve(prob, alg=DP5());

    # Generate the interactive plot
    @manipulate for i in widget(1:1:length(sol.t), label="", readout=false)
    plot(floors, sol.u[i][1:N], legend=false);
    title!("Time: $(round(sol.t[i], digits = 1)) (s)");
    xlabel!(L"\textrm{Floor}");
    ylabel!(L"\textrm{Displacement (m)}");
    yticks!([-0.2:0.1:0.2;]);
    ylims!(-0.2, 0.2);
    xlims!(0, Inf);
    end
end
```

In [2]:
function burj_khalifa(parameters, initialConditions, timespan, boundaryCondition, forcing)
    ω, γ, N, B, Ω = parameters; # Define the parameters
    N = convert(Integer, N); # Convert to an Integer
    floors = collect(1:1:N);
        
    A = diagm(0=>2*ones(N), 1=>ones(N-1), -1=>ones(N-1)); # Generate a tri-diagonal matrix
    # Set the boundary conditions
    A[1, 1] = boundaryCondition[1] == true ? 1 : A[1, 1]; # For the first mass
    A[end, end] = boundaryCondition[2] == true ? 1 : A[end, end]; # For the last mass
    
    global L = [zero(A) one(A); -ω^2*A -γ*A] |> sparse; # Generate L and make it sparse to increase speed
    forcing_function(t) =  ω^2*B*sin(Ω*t)+γ*Ω*B*cos(Ω*t); # Forcing function
    F(t) = [zeros(N); forcing_function(t); zeros(N-1)]; # Forcing function vector
    # Model our system and solve it, if there is a driving force, then add it
    function f(du, u, p, t) 
        mul!(du, L, u);
        du .+= forcing == true ? F(t) : 0;
    end
    prob = ODEProblem(f, initialConditions, timespan);
    sol = solve(prob, alg=DP5());

    # Generate the interactive plot
    @manipulate for i in widget(1:1:length(sol.t), label="", readout=false)
    plot(floors, sol.u[i][1:N], legend=false);
    title!("Time: $(round(sol.t[i], digits = 1)) (s)");
    xlabel!(L"\textrm{Floor}");
    ylabel!(L"\textrm{Displacement (m)}");
    yticks!([-0.2:0.1:0.2;]);
    ylims!(-0.2, 0.2);
    xlims!(0, Inf);
    end
    end;

# Equations of motion: Hooke's Law
The complete model of our system without a driving force is:
<div class="alert alert-block alert-warning">
<font color=blue>
\begin{align}
v_n&=\dot{x_n}\\
\dot{v_n}&=-\omega^{2} (2x_n - x_{n+1} - x_{n-1}) - \gamma (2v_n - v_{n+1} - v_{n-1})
\end{align}
</font>
</div>

The end point masses in our system, $x_1$ and $x_N$, are allowed to move freely and only couple to one nearest neighbour, namely, $x_2$ and $x_{N-1}$. Then,

$$ 
{\bf X}(t)\equiv \begin{bmatrix}
\mathbf{x}(t)\\
\mathbf{v}(t)\\
\end{bmatrix}
$$

$$
{\bf L}\equiv\begin{bmatrix}
    0_N         & I_N  \\
    -\omega^2 A_N & -\gamma A_N
\end{bmatrix},
$$

$$\ale{A_N&=\begin{bmatrix}
    1  & -1 & 0   \\
    -1 &  2 & -1 & \ddots \\
    0  & -1 &  2 & -1 & \ddots\\
       & \ddots  & -1 & 2 & -1 &  0\\
       &         & \ddots & -1 &  2 & -1 \\
       &         &        &  0 & -1 & 1 
\end{bmatrix}
}$$

Hence, the linear equation of motion to be solved is:
$$\frac{d{\bf X}}{dt}={\bf L}{\bf X}$$

1. 
    The code to setup the functions required to solve the problem is (using ω = π and γ = 0):
    ```julia
    ω = π;
    γ = 0;
    A = diagm(0=>2*ones(N), 1=>ones(N-1), -1=>ones(N-1)); # Generate a tri-diagonal matrix
    A[1, 1] = A[end, end] = 1; # Set the boundary conditions
    L = [zero(A) one(A); -ω^2*A -γ*A] |> sparse; # Generate L and make it sparse to increase speed
    f(du, u, p, t) = mul!(du, L, u);
    prob = ODEProblem(f, initialConditions, timespan);
    sol = solve(prob, alg=DP5());
    ```


2.   
    An interactive plot can be generated (using the function from preliminaries) as below:
    ```julia
    ω, γ, N, B, Ω = parameters = [π, 0, 300, 0, 0]
    N = convert(Integer, N);

    # Initial conditions
    x0 = transpose([1.0 zeros(1, N - 1)]);
    v0 = transpose(zeros(1, N));
    u0 = [x0; v0];

    # Time interval
    timespan =  (0.0, 600.0);

    # Boundary Condition
    # First element sets the ground floor condition and the second element sets the top floor condition
    boundaryConditions = [true, true];

    burj_khalifa(parameters, u0, timespan, boundaryCondition, false)
    ```

In [3]:
ω, γ, N, B, Ω = parameters = [π, 0, 300, 0, 0]
N = convert(Integer, N);

# Initial conditions
x0 = transpose([1.0 zeros(1, N - 1)]);
v0 = transpose(zeros(1, N));
u0 = [x0; v0];

# Time interval
timespan =  (0.0, 600.0);

# Boundary Condition
# First element sets the ground floor condition and the second element sets the top floor condition
boundaryCondition = [true, true];

burj_khalifa(parameters, u0, timespan, boundaryCondition, false)

Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Scope(Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :label), Any[""], Dict{Symbol,Any}(:className => "interact ",:style => Dict{Any,Any}(:padding => "5px 10px 0px 10px")))], Dict{Symbol,Any}(:className => "interact-flex-row-left")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :input), Any[], Dict{Symbol,Any}(:max => 3697,:min => 1,:attributes => Dict{Any,Any}(:type => "range",Symbol("data-bind") => "numericValue: index, valueUpdate: 'input', event: {change: function (){this.changes(this.changes()+1)}}","orient" => "horizontal"),:step => 1,:className => "slider slider is-fullwidth",:style => Dict{Any,Any}()))], Dict{Symbol,Any}(:className => "interact-flex-row-center")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[], Dict{Symbol,Any}())], Dict{Symbol,Any}(:className => "interact-flex-row-right"))], Dict{Symbol,Any}(:className => "interact-flex-row interact-widget")), Dict{String,Tuple{Observables.AbstractObservable,Union{Nothing, Bool}}}("changes" => (Observable{Int64} with 1 listeners. Value:
0, nothing),"index" => (Observable{Any} with 2 listeners. Value:
1849, nothing)), Set(String[]), nothing, Asset[Asset("js", "knockout", "/Users/naveenmathew/.julia/packages/Knockout/IP1uR/src/../assets/knockout.js"), Asset("js", "knockout_punches", "/Users/naveenmathew/.julia/packages/Knockout/IP1uR/src/../assets/knockout_punches.js"), Asset("js", nothing, "/Users/naveenmathew/.julia/packages/InteractBase/9mFwe/src/../assets/all.js"), Asset("css", nothing, "/Users/naveenmathew/.julia/packages/InteractBase/9mFwe/src/../assets/style.css"), Asset("css", nothing, "/Users/naveenmathew/.julia/packages/Interact/SbgIk/src/../assets/bulma_confined.min.css")], Dict{Any,Any}("changes" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"changes\"]()) ? (this.valueFromJulia[\"changes\"]=true, this.model[\"changes\"](val)) : undefined})")],"index" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"index\"]()) ? (this.valueFromJulia[\"index\"]=true, this.model[\"index\"](val)) : undefined})")]), WebIO.ConnectionPool(Channel{Any}(sz_max:32,sz_curr:0), Set(AbstractConnection[]), Base.GenericCondition{Base.AlwaysLockedST}(Base.InvasiveLinkedList{Task}(Task (runnable) @0x000000012c5e8490, Task (runnable) @0x000000012c5e8490), Base.AlwaysLockedST(1))), WebIO.JSString[WebIO.JSString("function () {\n    var handler = (function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init: function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            });\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            });\n            ko.applyBindingsToNode(\n                element,\n                {\n                    value: stringified,\n                    valueUpdate: allBindings.get('valueUpdate'),\n                },\n                context,\n            );\n        }\n    };\n    var json_data = {\"formatted_vals\":[\"1\",\"2\",\"3\",\"4\",\"5\",\"6\",\"7\",\"8\",\"9\",\"10\",\"11\",\"12\",\"13\",\"14\",\"15\",\"16\",\"17\",\"18\",\"19\",\"20\",\"21\",\"22\",\"23\",\"24\",\"25\",\"26\",\"27\",\"28\",\"29\",\"30\",\"31\",\"32\",\"33\",\"34\",\"35\",\"36\",\"37\",\"38\",\"39\",\"4

3. The displacement at $t_0 = 0 \ \textrm{s}$ is $x_0 = 1 \ \textrm{m}$, and the displacement at $t_1 = 92.2 \ \textrm{s}$ is $x_1 \approx 0.106 \ \textrm{m}$, thus the change in the displacement, $\Delta x \approx 0.894 \ \textrm{m}$. Hence, the speed of the leading edge of the pulse as it travels along the mass chain, $c \approx 0.894\pi \approx \pi \ \textrm{m} \textrm{s}^{-1}$ - as expected.

# Driving and damping
With a driving force, $x_0(t)=B\sin\Omega t$, the linear equation of motion is:
$$\frac{d{\bf X}}{dt}={\bf L}{\bf X}+{\bf F}(t),$$

4. The ground floor is not a degree of freedom to solve for, it instead provides a driving force while the last floor is still a degree of freedom to solve for. Hence, the matrix $A_N$, and the vector representing the driving force, the forcing vector $\mathbf{F}(t)$ are:
$$\ale{A_N&=\begin{bmatrix}
    2  & -1 & 0   \\
    -1 &  2 & -1 & \ddots \\
    0  & -1 &  2 & -1 & \ddots\\
       & \ddots  & -1 & 2 & -1 &  0\\
       &         & \ddots & -1 &  2 & -1 \\
       &         &        &  0 & -1 & 1 
\end{bmatrix}
}$$

$$\ale{\mathbf{F}(t)&\equiv\begin{bmatrix}
    0 \\
    \vdots\\
    \omega^2 B \mathrm{sin}(\Omega t)+\gamma \Omega B \mathrm{cos}(\Omega t)\\
    \vdots\\
    0
\end{bmatrix}
}$$

4. (a)  The code below was used to solve and generate an interactive plot for the driven equations of motion with $\gamma$ = 0.05, B = 0.1, $\Omega$ = $\frac{1}{2} \pi$ and the initial condition that the chain is initially at rest with all the masses in their equilibrium position from $t_i = 0$ to $t_f = 600$.
    ```julia
    ω, γ, N, B, Ω = parameters = [π, 0.05, 300, 0.1, 0.5*π]
    N = convert(Integer, N);

    # Initial conditions
    x0 = transpose(zeros(1, N));
    v0 = transpose(zeros(1, N));
    u0 = [x0; v0];

    # Time interval
    timespan =  (0.0, 600.0);

    # Boundary Conditions
    boundaryConditions = [false, true];

    burj_khalifa(parameters, u0, timespan, boundaryConditions, true)
    ```

In [4]:
ω, γ, N, B, Ω = parameters = [π, 0.05, 300, 0.1, 0.5*π]
N = convert(Integer, N);

# Initial conditions
x0 = transpose(zeros(1, N));
v0 = transpose(zeros(1, N));
u0 = [x0; v0];

# Time interval
timespan =  (0.0, 600.0);

# Boundary Conditions
boundaryConditions = [false, true];

burj_khalifa(parameters, u0, timespan, boundaryConditions, true)

Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Scope(Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :label), Any[""], Dict{Symbol,Any}(:className => "interact ",:style => Dict{Any,Any}(:padding => "5px 10px 0px 10px")))], Dict{Symbol,Any}(:className => "interact-flex-row-left")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :input), Any[], Dict{Symbol,Any}(:max => 1870,:min => 1,:attributes => Dict{Any,Any}(:type => "range",Symbol("data-bind") => "numericValue: index, valueUpdate: 'input', event: {change: function (){this.changes(this.changes()+1)}}","orient" => "horizontal"),:step => 1,:className => "slider slider is-fullwidth",:style => Dict{Any,Any}()))], Dict{Symbol,Any}(:className => "interact-flex-row-center")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[], Dict{Symbol,Any}())], Dict{Symbol,Any}(:className => "interact-flex-row-right"))], Dict{Symbol,Any}(:className => "interact-flex-row interact-widget")), Dict{String,Tuple{Observables.AbstractObservable,Union{Nothing, Bool}}}("changes" => (Observable{Int64} with 1 listeners. Value:
0, nothing),"index" => (Observable{Any} with 2 listeners. Value:
935, nothing)), Set(String[]), nothing, Asset[Asset("js", "knockout", "/Users/naveenmathew/.julia/packages/Knockout/IP1uR/src/../assets/knockout.js"), Asset("js", "knockout_punches", "/Users/naveenmathew/.julia/packages/Knockout/IP1uR/src/../assets/knockout_punches.js"), Asset("js", nothing, "/Users/naveenmathew/.julia/packages/InteractBase/9mFwe/src/../assets/all.js"), Asset("css", nothing, "/Users/naveenmathew/.julia/packages/InteractBase/9mFwe/src/../assets/style.css"), Asset("css", nothing, "/Users/naveenmathew/.julia/packages/Interact/SbgIk/src/../assets/bulma_confined.min.css")], Dict{Any,Any}("changes" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"changes\"]()) ? (this.valueFromJulia[\"changes\"]=true, this.model[\"changes\"](val)) : undefined})")],"index" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"index\"]()) ? (this.valueFromJulia[\"index\"]=true, this.model[\"index\"](val)) : undefined})")]), WebIO.ConnectionPool(Channel{Any}(sz_max:32,sz_curr:0), Set(AbstractConnection[]), Base.GenericCondition{Base.AlwaysLockedST}(Base.InvasiveLinkedList{Task}(Task (runnable) @0x000000010b324b50, Task (runnable) @0x000000010b324b50), Base.AlwaysLockedST(1))), WebIO.JSString[WebIO.JSString("function () {\n    var handler = (function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init: function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            });\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            });\n            ko.applyBindingsToNode(\n                element,\n                {\n                    value: stringified,\n                    valueUpdate: allBindings.get('valueUpdate'),\n                },\n                context,\n            );\n        }\n    };\n    var json_data = {\"formatted_vals\":[\"1\",\"2\",\"3\",\"4\",\"5\",\"6\",\"7\",\"8\",\"9\",\"10\",\"11\",\"12\",\"13\",\"14\",\"15\",\"16\",\"17\",\"18\",\"19\",\"20\",\"21\",\"22\",\"23\",\"24\",\"25\",\"26\",\"27\",\"28\",\"29\",\"30\",\"31\",\"32\",\"33\",\"34\",\"35\",\"36\",\"37\",\"38\",\"39\",\"40

4. (b) The same driven equation of motion as above was solved but with the damping constant increased such that $\gamma = 0.3$ instead. The code below was used to produce an interactive plot.
    ```julia
    ω, γ, N, B, Ω = parameters = [π, 0.3, 300, 0.1, 0.5*π];
    N = convert(Integer, N);

    # Initial conditions
    x0 = transpose(zeros(1, N));
    v0 = transpose(zeros(1, N));
    u0 = [x0; v0];

    # Time interval
    timespan =  (0.0, 600.0);

    # Boundary Conditions
    boundaryConditions = [false, true];

    burj_khalifa(parameters, u0, timespan, boundaryConditions, true)
    ```

In [5]:
ω, γ, N, B, Ω = parameters = [π, 0.3, 300, 0.1, 0.5*π];
N = convert(Integer, N);

burj_khalifa(parameters, u0, timespan, boundaryConditions, true)

Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Scope(Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :label), Any[""], Dict{Symbol,Any}(:className => "interact ",:style => Dict{Any,Any}(:padding => "5px 10px 0px 10px")))], Dict{Symbol,Any}(:className => "interact-flex-row-left")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :input), Any[], Dict{Symbol,Any}(:max => 1422,:min => 1,:attributes => Dict{Any,Any}(:type => "range",Symbol("data-bind") => "numericValue: index, valueUpdate: 'input', event: {change: function (){this.changes(this.changes()+1)}}","orient" => "horizontal"),:step => 1,:className => "slider slider is-fullwidth",:style => Dict{Any,Any}()))], Dict{Symbol,Any}(:className => "interact-flex-row-center")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[], Dict{Symbol,Any}())], Dict{Symbol,Any}(:className => "interact-flex-row-right"))], Dict{Symbol,Any}(:className => "interact-flex-row interact-widget")), Dict{String,Tuple{Observables.AbstractObservable,Union{Nothing, Bool}}}("changes" => (Observable{Int64} with 1 listeners. Value:
0, nothing),"index" => (Observable{Any} with 2 listeners. Value:
711, nothing)), Set(String[]), nothing, Asset[Asset("js", "knockout", "/Users/naveenmathew/.julia/packages/Knockout/IP1uR/src/../assets/knockout.js"), Asset("js", "knockout_punches", "/Users/naveenmathew/.julia/packages/Knockout/IP1uR/src/../assets/knockout_punches.js"), Asset("js", nothing, "/Users/naveenmathew/.julia/packages/InteractBase/9mFwe/src/../assets/all.js"), Asset("css", nothing, "/Users/naveenmathew/.julia/packages/InteractBase/9mFwe/src/../assets/style.css"), Asset("css", nothing, "/Users/naveenmathew/.julia/packages/Interact/SbgIk/src/../assets/bulma_confined.min.css")], Dict{Any,Any}("changes" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"changes\"]()) ? (this.valueFromJulia[\"changes\"]=true, this.model[\"changes\"](val)) : undefined})")],"index" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"index\"]()) ? (this.valueFromJulia[\"index\"]=true, this.model[\"index\"](val)) : undefined})")]), WebIO.ConnectionPool(Channel{Any}(sz_max:32,sz_curr:0), Set(AbstractConnection[]), Base.GenericCondition{Base.AlwaysLockedST}(Base.InvasiveLinkedList{Task}(Task (runnable) @0x000000010b326890, Task (runnable) @0x000000010b326890), Base.AlwaysLockedST(1))), WebIO.JSString[WebIO.JSString("function () {\n    var handler = (function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init: function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            });\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            });\n            ko.applyBindingsToNode(\n                element,\n                {\n                    value: stringified,\n                    valueUpdate: allBindings.get('valueUpdate'),\n                },\n                context,\n            );\n        }\n    };\n    var json_data = {\"formatted_vals\":[\"1\",\"2\",\"3\",\"4\",\"5\",\"6\",\"7\",\"8\",\"9\",\"10\",\"11\",\"12\",\"13\",\"14\",\"15\",\"16\",\"17\",\"18\",\"19\",\"20\",\"21\",\"22\",\"23\",\"24\",\"25\",\"26\",\"27\",\"28\",\"29\",\"30\",\"31\",\"32\",\"33\",\"34\",\"35\",\"36\",\"37\",\"38\",\"39\",\"40

4. (c) Comparing the plots with damping constants $\gamma = 0.05$ and $\gamma = 0.3$ as in 4(a) and 4(b), we see that in 4(a) the force due to the earthquake causes a displacement of about equal magnitude in all the floors, the building is likely to collapse. In constrast, in 4(b) the force is mainly contained within the first 100 floors and the top floors are largely unaffected. Comparing the plots in 4(a) and 4(b) to the plot in question 2, we see that without damping, the whole building is going to oscillate wildly. In the case of an earthquake, it would be wise to pick to the building modelled in 4(b) as it might wobble but it won't topple.